#### Import libraries and load data

In [11]:
# Import libraries
import pandas as pd
import numpy as np

In [12]:
# Load Data
data = pd.read_csv("processed_data.txt", sep="|")
data.head()

,Gender,Age_Scaled,Salary_Transformed_Scaled,Purchased
0,1,-1.781797,-1.809254,0
1,1,-0.253587,-1.752550,0
2,0,-1.113206,-0.725701,0
3,0,-1.017692,-0.246493,0
4,1,-1.781797,0.311281,0


#### Define model features and inputs

In [14]:
# Define features
X = data[["Gender", "Age_Scaled", "Salary_Transformed_Scaled"]]

In [15]:
# Define target
y = data["Purchased"]

#### Divide data into testing and training sets

In [17]:
# Divide data into training and testing.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=35)

In [18]:
# Convert train and test data sets into tensors
import tensorflow as tf
tf.random.set_seed(35)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

#### Define model and parameters

In [20]:
# Define model and layers
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(3,)), 
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [21]:
# Define model attributes
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 161 (644.00 B)

 Trainable params: 161 (644.00 B)

 Non-trainable params: 0 (0.00 B)

#### Train and evaluate model

In [23]:
# Train model
model.fit(X_train, y_train, batch_size=5, epochs=30, verbose=1)

Epoch 1/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5691 - loss: 0.6771
Epoch 2/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7784 - loss: 0.5795
Epoch 3/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8206 - loss: 0.5117
Epoch 4/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8177 - loss: 0.4616
Epoch 5/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8270 - loss: 0.4249
Epoch 6/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8522 - loss: 0.3980
Epoch 7/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8645 - loss: 0.3772
Epoch 8/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8659 - loss: 0.3611
Epoch 9/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8659 - loss: 0.3480
Epoch 10/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8725 - loss: 0.3370
Epoch 11/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8689 - loss: 0.3275
Epoch 12/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8699 - lo

In [24]:
# Evaluate model
results = model.evaluate(X_test, y_test, batch_size=5)
print("Model testing accuracy is", round(results[1], 3), "\nModel testing loss is", round(results[0], 3))

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9414 - loss: 0.2029  
Model testing accuracy is 0.95 
Model testing loss is 0.209


In [47]:
# Predictions from testing data
y_pred = model.predict(X_test)

# Set theshold
threshold = 0.5
predictions = np.where(y_pred > threshold, 1, 0)

# Generate confusion matrix
tf.math.confusion_matrix(y_test, predictions)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[50,  1],
       [ 3, 26]])>

#### Generate predictions based on new data

In [50]:
# Import scaler object from "Preprocessing" notebook
%store -r scaler

In [52]:
# Function to make predictions from a set of input tensors
def predict_with_new_data(input):
    scaled_input = scaler.fit_transform(input)
    scaled_input_tensor = tf.convert_to_tensor(scaled_input, dtype=tf.float32)
    prediction = model.predict(tf.stack(scaled_input_tensor))
    return tf.round(prediction * 1000.0) / 1000.0

In [54]:
# Generate new, random data to test model
random_new_data = []

for i in np.arange(1,11):
    gender_random = np.random.choice([0, 1]) # Gender
    age_random = int(np.random.uniform(18, 65)) # Age
    salary_transformed_random = int(np.random.uniform(180, 400)) # Salary
    test_user_data = [gender_random, age_random, salary_transformed_random]
    random_new_data.append(test_user_data)

random_new_data

[[0, 24, 338],
 [1, 58, 216],
 [1, 48, 231],
 [0, 43, 300],
 [0, 46, 396],
 [0, 45, 234],
 [1, 18, 189],
 [0, 38, 388],
 [0, 37, 360],
 [0, 29, 285]]

In [56]:
# Run prediction on randomized new data
new_predictions = predict_with_new_data(random_new_data)
new_predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[0.012],
       [0.951],
       [0.54 ],
       [0.156],
       [0.812],
       [0.322],
       [0.   ],
       [0.637],
       [0.317],
       [0.005]], dtype=float32)>